In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('Data/transactions.csv')
data = data.drop(['Unnamed: 0', 'Unnamed: 7'], axis=1)
data['Date'] = pd.to_datetime(data['Date'])

In [3]:
data.head()

,payer,receiver,tran_text,Date,dow
0,abacbefabc833a96597289665268d99e58ee1b2e4550dd...,9c044a510cd7333bda6da6a2a364372974a0a8a573ac3f...,:Italy:,2019-01-21,Monday
1,f0bf2b3ec1598e6ab6ef15ea0251328ca3731e237bdb49...,abacbefabc833a96597289665268d99e58ee1b2e4550dd...,Heroics,2019-01-19,Saturday
2,abacbefabc833a96597289665268d99e58ee1b2e4550dd...,523c406e16c2856eefb065c2ace781a81a608b6e826ca4...,To regain Kosovo,2019-01-18,Friday
3,f0bf2b3ec1598e6ab6ef15ea0251328ca3731e237bdb49...,abacbefabc833a96597289665268d99e58ee1b2e4550dd...,Reading at a club,2019-01-18,Friday
4,abacbefabc833a96597289665268d99e58ee1b2e4550dd...,f0bf2b3ec1598e6ab6ef15ea0251328ca3731e237bdb49...,For the Culture,2019-01-17,Thursday


In [7]:
# The question is, how many different people are involved in this dataset?

num_payers = data['payer'].unique().shape
print("Number of Payers: ", num_payers[0])
num_receivers = data['receiver'].unique().shape
print("Number of Receivers: ", num_receivers[0])

Number of Payers:  245877
Number of Receivers:  243751


In [ ]:
# So as we can see, there's a lot people involved. 
# There's simply no way to graph something this large.
# So what we want to do is only consider the transactions 
# for the most active users.